# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt

# Import API key
from api_keys import geoapify_key


In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hithadhoo,-0.6000,73.0833,28.24,78,100,7.29,MV,1690260502
1,1,ilulissat,69.2167,-51.1000,5.01,93,39,1.54,GL,1690260502
2,2,port mathurin,-19.6833,63.4167,23.69,75,79,9.47,MU,1690260502
3,3,port-aux-francais,-49.3500,70.2167,3.28,65,100,7.97,TF,1690260502
4,4,macapa,0.0389,-51.0664,25.99,94,0,1.54,BR,1690260503


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
humidity_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.01,
    color = "City"
)


# Display the map
humidity_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [58]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df.loc[(city_data_df["Max Temp"] > 20) & (city_data_df["Max Temp"] < 25) & (city_data_df["Cloudiness"] < 10)]
ideal_weather = ideal_weather.dropna()
ideal_weather.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,igarka,67.4667,86.5833,22.90,52,3,2.67,RU,1690260506
59,59,roquetas de mar,36.7642,-2.6148,23.00,49,0,1.60,ES,1690260393
69,69,magnolia,33.2671,-93.2393,22.83,94,0,2.32,US,1690260513
124,124,lompoc,34.6391,-120.4579,23.21,91,0,2.06,US,1690260522
127,127,karratha,-20.7377,116.8463,22.39,47,0,10.02,AU,1690260523


### Step 3: Create a new DataFrame called `hotel_df`.

In [59]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(ideal_weather[["City","Country","Humidity","Lat","Lng"]])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
hotel_df


,City,Country,Humidity,Lat,Lng,Hotel Name
25,igarka,RU,52,67.4667,86.5833,
59,roquetas de mar,ES,49,36.7642,-2.6148,
69,magnolia,US,94,33.2671,-93.2393,
124,lompoc,US,91,34.6391,-120.4579,
127,karratha,AU,47,-20.7377,116.8463,
130,kununurra,AU,23,-15.7667,128.7333,
146,inverell,AU,47,-29.7833,151.1167,
159,palmer,US,80,42.1584,-72.3287,
161,brookings,US,82,44.3114,-96.7984,
178,saint-paul,RE,76,-21.0096,55.2707,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [60]:
# Set parameters to search for a hotel
limit = 20
radius = 1000

params = {
    "categories":"accommodation.hotel",
    "radius":radius,
    "apiKey":geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address_json = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
igarka - nearest hotel: No hotel found
roquetas de mar - nearest hotel: Pensión Juan Pedro
magnolia - nearest hotel: Loft on the Square
lompoc - nearest hotel: No hotel found
karratha - nearest hotel: Karratha International Hotel
kununurra - nearest hotel: Kununurra Country Club Resort
inverell - nearest hotel: No hotel found
palmer - nearest hotel: No hotel found
brookings - nearest hotel: No hotel found
saint-paul - nearest hotel: No hotel found
sao joao da barra - nearest hotel: No hotel found
trenton - nearest hotel: No hotel found
fereydunshahr - nearest hotel: No hotel found
tortosa - nearest hotel: Parador Nacional de Turisme de Tortosa
hanceville - nearest hotel: No hotel found
lewistown - nearest hotel: No hotel found
la ronge - nearest hotel: La Ronge Hotel and Suites
usinsk - nearest hotel: общежитие Лукойл-Коми
san carlos - nearest hotel: No hotel found
fort campbell north - nearest hotel: No hotel found
gurupi - nearest hotel: No hotel found
port hedl

,City,Country,Humidity,Lat,Lng,Hotel Name
25,igarka,RU,52,67.4667,86.5833,No hotel found
59,roquetas de mar,ES,49,36.7642,-2.6148,Pensión Juan Pedro
69,magnolia,US,94,33.2671,-93.2393,Loft on the Square
124,lompoc,US,91,34.6391,-120.4579,No hotel found
127,karratha,AU,47,-20.7377,116.8463,Karratha International Hotel
130,kununurra,AU,23,-15.7667,128.7333,Kununurra Country Club Resort
146,inverell,AU,47,-29.7833,151.1167,No hotel found
159,palmer,US,80,42.1584,-72.3287,No hotel found
161,brookings,US,82,44.3114,-96.7984,No hotel found
178,saint-paul,RE,76,-21.0096,55.2707,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [62]:
%%capture --no-display

# Configure the map plot
hotel_info = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    tiles = "OSM",
    hover_cols=["Hotel Name", "Country"],
    size="Humidity",
    color = "City",
    frame_width =800,
    frame_height =600,
    alpha= 0.8
    )

# Display the map
display(hotel_info)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)